# Modeling definitions
 * **Binary Predictions** : For high-level model exploration, standard metrics including AUC of ROC 
   - Using scipy.--model--.predict
 * **Continuous Predictions** : Given a model, for feture engineering, Net Reclassification Index (NRI)
   - Using scipy.--model--.predict_proba

## Continuous Probabilities per class

In [39]:
def check_cat(prob,thresholds):
    cat=0
    for i,v in enumerate(thresholds):
        if prob>v:
            cat=i
    return cat

def make_cat_matrix(ref, new, indices, thresholds):
    num_cats=len(thresholds)
    mat=np.zeros((num_cats,num_cats))
    for i in indices:
        row,col=check_cat(ref[i],thresholds),check_cat(new[i],thresholds)
        mat[row,col]+=1
    return mat
        
def nri(y_truth,y_ref, y_new,risk_thresholds):
    event_index = np.where(y_truth==1)[0]
    nonevent_index = np.where(y_truth==0)[0]
    event_mat=make_cat_matrix(y_ref,y_new,event_index,risk_thresholds)
    nonevent_mat=make_cat_matrix(y_ref,y_new,nonevent_index,risk_thresholds)
#     events_up, events_down = event_mat[0,1:].sum()+event_mat[1,2:].sum()+event_mat[2,3:].sum(),event_mat[1,:1].sum()+event_mat[2,:2].sum()+event_mat[3,:3].sum()
#     nonevents_up, nonevents_down = nonevent_mat[0,1:].sum()+nonevent_mat[1,2:].sum()+nonevent_mat[2,3:].sum(),nonevent_mat[1,:1].sum()+nonevent_mat[2,:2].sum()+nonevent_mat[3,:3].sum()
    events_up, events_down = event_mat[0,1],event_mat[1,0]
    nonevents_up, nonevents_down = nonevent_mat[0,1],nonevent_mat[1,0]
    nri_events = (events_up/len(event_index))-(events_down/len(event_index))
    nri_nonevents = (nonevents_down/len(nonevent_index))-(nonevents_up/len(nonevent_index))
    return nri_events, nri_nonevents, nri_events + nri_nonevents 


## Binary predictions

In [48]:
from sklearn.metrics import confusion_matrix, roc_auc_score
import compare_auc_delong_xu
from scipy import stats

def get_auc_ci(auc, auc_cov, alpha=0.95) :
    alpha = .95
    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

    ci = stats.norm.ppf(
        lower_upper_q,
        loc=auc,
        scale=auc_std)

    ci[ci > 1] = 1
    return ci
  
def cv_scorer(clf, X, y):
    y_pred = clf.predict(X)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    sen = (tp)/(tp+fn)
    sp = (tn)/(tn+fp)
    ppv = (tp)/(tp+fp)
    npv = (tn)/(tn+fn)
    f1 = 2*(sen*ppv)/(sen+ppv)
    auc = round(roc_auc_score(y, y_pred), 3)

    return {
                'acc': round(acc, 3)*100,
                'sen' : round(sen, 3)*100,
                'spe' : round(sp, 3)*100,
                'ppv' : round(ppv, 3)*100,
                'npv' :  round(npv, 3)*100,
                'auc' : round(auc, 2), 
                'nri' : 1
    }  

def add_mean_metrics(scorer) :
    metrics = ['acc', 'sen', 'spe', 'ppv', 'npv', 'nri']
    for m in metrics :
        scorer[m] = round(scorer["test_"+m].mean(), 1)
        scorer[m+"_err"] = round((scorer["test_"+m].std())/np.sqrt(scorer["test_"+m].size), 2)
    m = 'auc'
    scorer[m] = round(scorer["test_"+m].mean(), 2)
    scorer[m+"_err"] = round((scorer["test_"+m].std())/np.sqrt(scorer["test_"+m].size), 2)
    return scorer

def extract_metrics(y, y_pred, y_pred_prob=None, y_pred_prob_ref=None):
#     tp, fn, fp, tn = confusion_matrix(y, y_pred).ravel()
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
#     if debug: print('TN : {}, FN : {}, FP : {}, TP : {}'.format(tn, fp, fn, tp))
    acc = (tp+tn)/(tp+tn+fp+fn)
    sen = (tp)/(tp+fn)
    sp = (tn)/(tn+fp)
    ppv = (tp)/(tp+fp)
    npv = (tn)/(tn+fn)
    f1 = 2*(sen*ppv)/(sen+ppv)
#    fpr = (fp)/(fp+tn)
#    tpr = (tp)/(tp+fn)
    auc = round(roc_auc_score(y, y_pred), 3)
    auc_delong, auc_cov = compare_auc_delong_xu.delong_roc_variance(y, y_pred)
    auc_std = np.sqrt(auc_cov)
    auc_ci = get_auc_ci(auc_delong, auc_cov)
    bp = (tp + fn) / (tp+tn+fp+fn)
    
    if y_pred_prob is None or y_pred_prob_ref is None :
        ev, nonev, tot = 1, 1, 2
    else :
        
        ev, nonev, tot = nri(y,y_pred_prob_ref[:,1],y_pred_prob[:,1],[0.02,0.5])

    return {
                'acc': round(acc, 3)*100,
                'sen' : round(sen, 3)*100,
                'spe' : round(sp, 3)*100,
                'ppv' : round(ppv, 3)*100,
                'npv' :  round(npv, 3)*100,
                'auc' : round(auc, 2), 
#                 'AUC' : round(auc_delong, 2), 
                'auc_err' : round(auc_std, 2),
                'nri' : round(tot, 2), 
                # 'NRI (E)' : ev, 
                # 'NRI (NE)' : nonev, 
#         '{}[{}]'.format(auc_delong, str(auc_ci)),
#                 'AUC' : auc,
                'bp' : round(bp, 3)*100
        
    }  
# ['Accuracy(%)', 'Sensitivity(%)', 'Specificity(%)', 'PPV(%)', 'NPV(%)', 'AUC','AUC(err)','NRI','Base prevalance(%)'] 


## Metrics Class

In [41]:
class OverallMetrics:
    def __init__(self):
        self.df = pd.DataFrame()

    def getDF(self) :
        return self.df
    
    def addMetrics(self, metrics_record, index_values):
        for idx in index_values :
            metrics_record[idx] = index_values[idx]
        metrics = pd.DataFrame.from_records([metrics_record])#, 
                                        # index=[[key] for key in keys])
        self.df  = pd.concat([self.df, metrics])
    
class MetricsByFeatureAddition:
    def __init__(self):
        self.df = pd.DataFrame()

    def getDF(self) :
        return self.df
    
    def addMetrics(self, metrics_record, index_values):
        metrics_df = pd.DataFrame.from_records([metrics_record])
        _by_metric = pd.Series(metrics_df.mean(axis=0), name='ALL').reset_index(name='value').rename(columns = {'index' : 'metric'})
        for idx in index_values :
            _by_metric[idx] = index_values[idx]
        self.df  = pd.concat([self.df, _by_metric])


**Encoding category fields**

In [42]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()

def encodeCatsAndNormalize(df) :
    encodeFields = ["PTGENDER", "ETHNICRACE", 
                   #"PTETHCAT", "PTRACCAT",
                   "PTMARRY"]
    df_fields = df.columns
    for ef in encodeFields :
        if ef in df_fields:
            df[ef] = ord_enc.fit_transform(df[[ef]])
    return df


## Train and Test - One Model

 - Training : CD+AD
     - Metrics : fit and predict on training data
 - Prediction : MCI

In [43]:
def fit_and_predict(clf, X, y, clf_ref=None) :
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123)        
    clf.fit(x_train, y_train)
#     clf.fit(normed_data, y)
    print('OOB Score: ', clf.oob_score_)
    y_pred = clf.predict(x_test)
#     y_pred = clf.predict(normed_data)
    if clf_ref is not None:
        test_pred_prob=clf.predict_proba(x_test[curr_model_features])
    #             test_pred_prob=clf.predict_proba(normed_data[curr_model_features])
        clf_ref = outputs_by_model[-1]["model"] #outputs_by_model[i-1]
        test_pred_prob_ref=clf_ref.predict_proba(x_test[ref_model_features])
        return extract_metrics(y_test, y_pred, test_pred_prob, test_pred_prob_ref)
    else :
        return extract_metrics(y_test, y_pred)


## Model training and testing - ALL Features

In [44]:
def trainandtest(supertrain, feature_cols, train_ycol, model_label, ALLMETRICS, model_type='ALL') :
    print(feature_cols)
    training_metrics = ALLMETRICS["train"]["all"]
    metrics_by_feature_addition = ALLMETRICS["train"]["feature_addition"]
    final_metrics_ = ALLMETRICS["test"]["all"]
    MCI_metrics_by_feature_addition = ALLMETRICS["test"]["feature_addition"]
    
    ###TRAIN
    train_features = feature_cols + [train_ycol]
    adm = supertrain[train_features].dropna()
    adm = encodeCatsAndNormalize(adm)

    N_train = adm.shape[0]
    X = adm[feature_cols]
    y = adm[train_ycol]

    # Transform data
    sc = StandardScaler()
    normed_data = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

    clf=RandomForestClassifier(oob_score=True)
#     x_train, x_test, y_train, y_test = train_test_split(normed_data, y, test_size=0.30, random_state=123)        
#     clf.fit(x_train, y_train)
# #     clf.fit(normed_data, y)
#     print('OOB Score: ', clf.oob_score_)cv
#     y_pred = clf.predict(x_test)
#     y_pred = clf.predict(normed_data)
    
#     training_metrics.addMetrics(extract_metrics(y_test, y_pred),
    training_metrics.addMetrics(fit_and_predict(clf, normed_data, y),
#     training_metrics.addMetrics(extract_metrics(y, y_pred),
                                {'model' : model_label,  
                                 'feature': model_type, 
                                 'count': '{}({}%)'.format(str(N_train), str(round(100 * y.sum()/y.shape[0], 2)))}
                               )

    clf=RandomForestClassifier(oob_score=True)
    clf.fit(normed_data, y)
    feature_imp = pd.Series(clf.feature_importances_, index=X.columns, name=model_label).sort_values(ascending=False)
#     metrics_by_feature_addition.addMetrics(extract_metrics(y, y_pred),
#     metrics_by_feature_addition.addMetrics(extract_metrics(y_test, y_pred),
    metrics_by_feature_addition.addMetrics(fit_and_predict(clf, normed_data, y),
                      {'model' : model_label,
                      'feature' : model_type,
                      'feature_weight' : 1})
    model_name = '{}, N (%AD)= {}({}%)'.format(model_label, str(N_train), str(round(100 * y.sum()/y.shape[0], 2)))

    ### Prediction on MCI
    test_features = feature_cols + [test_ycol]
    mci_datasets = [{'key' : 'ALL', 'data' : ADM_MCI[test_features].dropna()}, 
                    {'key' : 'AB+', 'data' : ADM_MCI[ADM_MCI["ABP"] == 1][test_features].dropna()},
                    {'key' : 'AB-', 'data' : ADM_MCI[ADM_MCI["ABP"] == 0][test_features].dropna()},
                   ]  
    for mci_dataset in mci_datasets:
        mci_df = mci_dataset['data']
        mci_test_X = mci_df[feature_cols]
        mci_test_X = encodeCatsAndNormalize(mci_test_X)
        MCI_abp_true = mci_df[test_ycol]
        N = mci_df[test_ycol].shape[0]
        BP = 100 * (mci_df[mci_df[test_ycol] == 1].shape[0])/N
        # Transform data
        sc_mci = StandardScaler()
        normed_data_mci = pd.DataFrame(sc_mci.fit_transform(mci_test_X), columns = mci_test_X.columns)
        y_abp_pred = clf.predict(normed_data_mci)
        final_metrics_.addMetrics(extract_metrics(MCI_abp_true, y_abp_pred, True),
                                {'dataset': mci_dataset['key'], 'model' : model_label,  'feature': model_type, 'count': '{}'.format(N)}
                                   )

        MCI_metrics_by_feature_addition.addMetrics(extract_metrics(MCI_abp_true, y_abp_pred),
                      {'model' : model_label,
                      'feature' : model_type,
                      'feature_weight' : 1,
                      'dataset' : mci_dataset['key']})
    
    ALLMETRICS = {"train" : {
                    "all" : training_metrics,
                    "feature_addition" : metrics_by_feature_addition
                 },
                "test": {
                    "all" : final_metrics_,
                    "feature_addition" : MCI_metrics_by_feature_addition
                 }
            }

    return clf, feature_imp, ALLMETRICS


## Model training and testing - ALL Features - CV

In [49]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import permutation_importance

class TunedRF:
    def __init__(self, scoring):
        self.model = RandomForestClassifier(random_state= 42)
        self.cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
        self.grid = dict()
        self.grid['n_estimators'] = np.array([50, 100, 150, 200, 250, 300, 350])
        self.search = GridSearchCV(RandomForestClassifier(random_state= 42), self.grid, scoring=scoring, 
                                  refit="auc", 
                                  return_train_score=True,
                              cv=self.cv, n_jobs=-1)
    
    def getGSObj(self):
        return self.search
    
    def getBestEstimatorMetrics(self) :
        metrics = pd.DataFrame(self.search.cv_results_).loc[self.search.best_index_]
        mdf = {}
        metrics_of_interest = ['acc', 'sen', 'spe', 'ppv', 'npv', 'auc', 'nri']
        for moi in metrics_of_interest:
            mdf[moi] = metrics['mean_test_'+moi]
            mdf[moi+'_err'] = metrics['std_test_'+moi]
        return mdf
        
    def getFeatureImportances(self, X, y) :
        print('Feature importance by permutation...')
        skf = StratifiedKFold(n_splits=5)
        fis = []
        for train_index, test_index in skf.split(X, y):
            clf = self.getTunedModel()
            X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            clf.fit(X_train, y_train)
            r = permutation_importance(clf, X_test, y_test,
                           n_repeats=30,
                           random_state=0, scoring='roc_auc')
            fis.append(r.importances_mean)
        return np.array(fis).mean(axis=0)
            
    def getBestFitModel(self, X, y) :
        self.search.fit(X, y)
#         print('Config of Tuned RF : %s' % self.search.best_params_)
        return self.search.best_estimator_, self.getBestEstimatorMetrics() 
    
    def getTunedModel(self) :
        return RandomForestClassifier(random_state= 42, \
                                      n_estimators=self.search.best_params_['n_estimators'])

from sklearn.model_selection import RepeatedStratifiedKFold

def getFeatureImportances(X, y) :
    print('Feature importance by permutation...')
    skf = StratifiedKFold(n_splits=5)#, n_repeats=5)
    fis = []
    for train_index, test_index in skf.split(X, y):
        clf = RandomForestClassifier(oob_score=True)
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf.fit(X_train, y_train)
        r = permutation_importance(clf, X_test, y_test,
                       n_repeats=30,
                       random_state=123, scoring='roc_auc')
        fis.append(r.importances_mean)
    return np.array(fis).mean(axis=0)

def trainandtest_CV(supertrain, feature_cols, train_ycol, model_label, ALLMETRICS, model_type='ALL', ft_eng=True) :
    print(feature_cols)
    training_metrics = ALLMETRICS["train"]["all"]
    metrics_by_feature_addition = ALLMETRICS["train"]["feature_addition"]
    final_metrics_ = ALLMETRICS["test"]["all"]
    MCI_metrics_by_feature_addition = ALLMETRICS["test"]["feature_addition"]
    
    ###TRAIN
    train_features = feature_cols + [train_ycol]
    adm = supertrain[train_features].dropna()
    adm = encodeCatsAndNormalize(adm)

    # Shuffle the dataset
#     adm = adm.sample(frac = 1).reset_index().drop(['index'], axis=1)
    N_train = adm.shape[0]
    X = adm[feature_cols]
    y = adm[train_ycol]

    # Transform data
    sc = StandardScaler()
    normed_data = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

    clf=RandomForestClassifier(oob_score=True)
#     clf = TunedRF(cv_scorer)
#     best_fit_model, metrics = clf.getBestFitModel(normed_data, y)
#     tuned_clf = clf.getTunedModel()
    cv_results = cross_validate(clf, X, y, cv=5,
                             scoring=cv_scorer
#                                 , return_estimator=True
                               )
#     getFeatureImportances(normed_data, y)
    validation_metrics = add_mean_metrics(cv_results) 
    feature_imp = None
    if ft_eng :
        mean_ft_imp = getFeatureImportances(normed_data, y) 
        feature_imp = pd.Series(mean_ft_imp, index=X.columns, name=model_label).sort_values(ascending=False)
    #mean_ft_imp = np.array([c.feature_importances_ for c in cv_results['estimator']]).mean(axis=0)
    training_metrics.addMetrics(validation_metrics,
                                {'model' : model_label,  
                                 'feature': model_type, 
                                 'count': '{}({}%)'.format(str(N_train), str(round(100 * y.sum()/y.shape[0], 2)))}
                               )

    metrics_by_feature_addition.addMetrics(validation_metrics,
                      {'model' : model_label,
                      'feature' : model_type,
                      'feature_weight' : 1})
    
    clf.fit(normed_data, y)
#     model_ft_imps = clf.getFeatureImportances(normed_data, y)
#     feature_imp = pd.Series(clf.feature_importances_, index=X.columns, name=model_label).sort_values(ascending=False)
#     feature_imp = pd.Series(model_ft_imps, index=X.columns, name=model_label).sort_values(ascending=False)
    model_name = '{}, N (%AD)= {}({}%)'.format(model_label, str(N_train), str(round(100 * y.sum()/y.shape[0], 2)))

    ### Prediction on MCI
    test_features = feature_cols + [test_ycol]
    mci_datasets = [{'key' : 'ALL', 'data' : ADM_MCI[test_features].dropna()}, 
                    {'key' : 'AB+', 'data' : ADM_MCI[ADM_MCI["ABP"] == 1][test_features].dropna()},
                    {'key' : 'AB-', 'data' : ADM_MCI[ADM_MCI["ABP"] == 0][test_features].dropna()},
                   ]  
    for mci_dataset in mci_datasets:
        mci_df = mci_dataset['data']
        mci_test_X = mci_df[feature_cols]
        mci_test_X = encodeCatsAndNormalize(mci_test_X)
        MCI_abp_true = mci_df[test_ycol]
        N = mci_df[test_ycol].shape[0]
        BP = 100 * (mci_df[mci_df[test_ycol] == 1].shape[0])/N
        # Transform data
        sc_mci = StandardScaler()
        normed_data_mci = pd.DataFrame(sc_mci.fit_transform(mci_test_X), columns = mci_test_X.columns)
        y_abp_pred = clf.predict(normed_data_mci)
        final_metrics_.addMetrics(extract_metrics(MCI_abp_true, y_abp_pred, True),
                                {'dataset': mci_dataset['key'], 'model' : model_label,  'feature': model_type, 'count': '{}'.format(N)}
                                   )

        MCI_metrics_by_feature_addition.addMetrics(extract_metrics(MCI_abp_true, y_abp_pred),
                      {'model' : model_label,
                      'feature' : model_type,
                      'feature_weight' : 1,
                      'dataset' : mci_dataset['key']})
    
    ALLMETRICS = {"train" : {
                    "all" : training_metrics,
                    "feature_addition" : metrics_by_feature_addition
                 },
                "test": {
                    "all" : final_metrics_,
                    "feature_addition" : MCI_metrics_by_feature_addition
                 }
            }

    return clf, ALLMETRICS, feature_imp 


## Feature Engineering

In [50]:
def feature_engineer(supertrain, feature_cols, train_ycol, model_label, ALLMETRICS) :
    metrics_by_feature_addition = ALLMETRICS["train"]["feature_addition"]
    MCI_metrics_by_feature_addition = ALLMETRICS["test"]["feature_addition"]
    
    ###TRAIN
    train_features = feature_cols + [train_ycol]
    adm = supertrain[train_features].dropna()
    adm = encodeCatsAndNormalize(adm)
    # Shuffle the dataset
    adm = adm.sample(frac = 1, random_state=123).reset_index().drop(['index'], axis=1)
    N_train = adm.shape[0]
    X = adm[feature_cols]
    y = adm[train_ycol]
    # Transform data
    sc = StandardScaler()
    normed_data = pd.DataFrame(sc.fit_transform(X), columns = X.columns)
    x_train, x_test, y_train, y_test = train_test_split(normed_data, y, test_size=0.30, random_state=123)        

    fields_by_weight = feature_imp.index
    outputs_by_model = []
    print('Feature engineering : ', end='')
    fields_final_model = []
    validation_metrics_ref = None
    for i in range(len(fields_by_weight)) :
        print(fields_by_weight[i]+', ', end='')
        ref_model_features = []
        if len(outputs_by_model) > 0:
            ref_model_features = outputs_by_model[-1]["model_features"]
        curr_model_features = ref_model_features + [fields_by_weight[i]]
            
        incremental_fields = fields_by_weight[:i+1]
        inc_field = fields_by_weight[i]
        feature_weight = round(feature_imp.loc[inc_field], 3)
        features_label = ('' if i == 0 else '+') + inc_field 

        clf=RandomForestClassifier()
        count = 1
        clf.fit(x_train[curr_model_features], y_train)
#         clf.fit(normed_data[curr_model_features], y)
        y_pred = clf.predict(x_test[curr_model_features])
        # metrics = pd.DataFrame.from_records([extract_metrics(y_test, y_pred)], index=[count])
        test_pred_prob, test_pred_prob_ref = None, None
        # saveMetrics = False
        ### For NRI
        if i > 0 :
            ### NRI
            test_pred_prob=clf.predict_proba(x_test[curr_model_features])
#             test_pred_prob=clf.predict_proba(normed_data[curr_model_features])
            clf_ref = outputs_by_model[-1]["model"] #outputs_by_model[i-1]
            test_pred_prob_ref=clf_ref.predict_proba(x_test[ref_model_features])
#             test_pred_prob_ref=clf_ref.predict_proba(normed_data[ref_model_features])

#         validation_metrics = extract_metrics(y_test, y_pred, test_pred_prob, test_pred_prob_ref)
        validation_metrics = extract_metrics(y_test, y_pred, test_pred_prob, test_pred_prob_ref)
        addFeature = (i > 0 and (validation_metrics["auc"] - validation_metrics_ref["auc"] >= 0.01) )
#         if validation_metrics["NRI"] > 0 :
        if i==0 or addFeature :
            test_features = feature_cols + [test_ycol]
            mci_datasets = [{'key' : 'ALL', 'data' : ADM_MCI[test_features].dropna()}, 
                            {'key' : 'AB+', 'data' : ADM_MCI[ADM_MCI["ABP"] == 1][test_features].dropna()},
                            {'key' : 'AB-', 'data' : ADM_MCI[ADM_MCI["ABP"] == 0][test_features].dropna()},
                           ]  
            for mci_dataset in mci_datasets:
                mci_df = mci_dataset['data']
                mci_test_X = mci_df[feature_cols]
                mci_test_X = encodeCatsAndNormalize(mci_test_X)
                MCI_abp_true = mci_df[test_ycol]
                N = mci_df[test_ycol].shape[0]
                BP = 100 * (mci_df[mci_df[test_ycol] == 1].shape[0])/N
                # Transform data
                sc_mci = StandardScaler()
                normed_data_mci = pd.DataFrame(sc_mci.fit_transform(mci_test_X), columns = mci_test_X.columns)
                ###MCI Test
                y_abp_pred = clf.predict(normed_data_mci[curr_model_features])
                mci_test_pred_prob, mci_test_pred_prob_ref = None, None
                if i > 0 :
                    ### NRI
                    mci_test_pred_prob=clf.predict_proba(normed_data_mci[curr_model_features])
                    clf_ref = outputs_by_model[-1]["model"] #outputs_by_model[i-1]
                    mci_test_pred_prob_ref=clf_ref.predict_proba(normed_data_mci[ref_model_features])
                # metrics_mci = pd.DataFrame.from_records([extract_metrics(MCI_abp_true, y_abp_pred)], index=[count])

                ###MCI
                MCI_metrics_by_feature_addition.addMetrics(
                    extract_metrics(MCI_abp_true, y_abp_pred, mci_test_pred_prob, mci_test_pred_prob_ref), #Add NRI flag
                      {'model' : model_label,
                      'feature' : features_label,
                      'feature_weight' : feature_weight,
                      'dataset' : mci_dataset['key']})
            fields_final_model.append(fields_by_weight[i])
            outputs_by_model.append({"model" : clf, "model_features" : curr_model_features})
            metrics_by_feature_addition.addMetrics(validation_metrics, #Add NRI flag
                      {'model' : model_label,
                      'feature' : features_label,
                      'feature_weight' : feature_weight})
            validation_metrics_ref = validation_metrics
            
    ALLMETRICS["train"]["feature_addition"] = metrics_by_feature_addition
    ALLMETRICS["test"]["feature_addition"] = MCI_metrics_by_feature_addition

    return fields_final_model, ALLMETRICS



## Feature Engineering - CV

In [47]:
def feature_engineer_CV(supertrain, feature_cols, train_ycol, model_label, ALLMETRICS) :
    metrics_by_feature_addition = ALLMETRICS["train"]["feature_addition"]
    MCI_metrics_by_feature_addition = ALLMETRICS["test"]["feature_addition"]
    
    ###TRAIN
    train_features = feature_cols + [train_ycol]
    adm = supertrain[train_features].dropna()
    adm = encodeCatsAndNormalize(adm)
    # Shuffle the dataset
#     adm = adm.sample(frac = 1, random_state=123).reset_index().drop(['index'], axis=1)
    N_train = adm.shape[0]
    X = adm[feature_cols]
    y = adm[train_ycol]
    # Transform data
    sc = StandardScaler()
    normed_data = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

    fields_by_weight = feature_imp.index
    outputs_by_model = []
    print('Feature engineering : ', end='')
    fields_final_model = []
    validation_metrics_ref = None
    for i in range(len(fields_by_weight)) :
        print(fields_by_weight[i]+', ', end='')
        ref_model_features = []
        if len(outputs_by_model) > 0:
            ref_model_features = outputs_by_model[-1]["model_features"]
        curr_model_features = ref_model_features + [fields_by_weight[i]]
            
        incremental_fields = fields_by_weight[:i+1]
        inc_field = fields_by_weight[i]
        feature_weight = round(feature_imp.loc[inc_field], 3)
        features_label = ('' if i == 0 else '+') + inc_field 

        count = 1
#         clf = TunedRF(cv_scorer)
#         best_fit_model, metrics = clf.getBestFitModel(normed_data[curr_model_features], y)
#         tuned_clf = clf.getTunedModel()
        clf=RandomForestClassifier(oob_score=True)
        cv_results = cross_validate(clf, normed_data[curr_model_features], y, cv=5,
                                 scoring=cv_scorer)
        validation_metrics = add_mean_metrics(cv_results)
        addFeature = (i > 0 and (validation_metrics["auc"] - validation_metrics_ref["auc"] >= 0.01) )
        if i==0 or addFeature :
            clf=RandomForestClassifier(oob_score=True)
            clf.fit(normed_data[curr_model_features], y)
            test_features = feature_cols + [test_ycol]
            mci_datasets = [{'key' : 'ALL', 'data' : ADM_MCI[test_features].dropna()}, 
                            {'key' : 'AB+', 'data' : ADM_MCI[ADM_MCI["ABP"] == 1][test_features].dropna()},
                            {'key' : 'AB-', 'data' : ADM_MCI[ADM_MCI["ABP"] == 0][test_features].dropna()},
                           ]  
            for mci_dataset in mci_datasets:
                mci_df = mci_dataset['data']
                mci_test_X = mci_df[feature_cols]
                mci_test_X = encodeCatsAndNormalize(mci_test_X)
                MCI_abp_true = mci_df[test_ycol]
                N = mci_df[test_ycol].shape[0]
                BP = 100 * (mci_df[mci_df[test_ycol] == 1].shape[0])/N
                # Transform data
                sc_mci = StandardScaler()
                normed_data_mci = pd.DataFrame(sc_mci.fit_transform(mci_test_X), columns = mci_test_X.columns)
                ###MCI Test
                y_abp_pred = clf.predict(normed_data_mci[curr_model_features])
                ###MCI
                MCI_metrics_by_feature_addition.addMetrics(
                    extract_metrics(MCI_abp_true, y_abp_pred),
                      {'model' : model_label,
                      'feature' : features_label,
                      'feature_weight' : feature_weight,
                      'dataset' : mci_dataset['key']})
            fields_final_model.append(fields_by_weight[i])
            outputs_by_model.append({"model" : clf, "model_features" : curr_model_features})
            metrics_by_feature_addition.addMetrics(validation_metrics,
                      {'model' : model_label,
                      'feature' : features_label,
                      'feature_weight' : feature_weight})
            validation_metrics_ref = validation_metrics
            
    ALLMETRICS["train"]["feature_addition"] = metrics_by_feature_addition
    ALLMETRICS["test"]["feature_addition"] = MCI_metrics_by_feature_addition

    return fields_final_model, ALLMETRICS



## McNemar test

In [ ]:
from mlxtend.evaluate import mcnemar_table, mcnemar

def mcnemar_test(supertrain, feature_cols, final_features, train_ycol, test_ycol) :
    print(feature_cols)
    ###TRAIN
    train_features = feature_cols + [train_ycol]
    adm_full = supertrain[train_features].dropna()
    adm = encodeCatsAndNormalize(adm_full)
    # Transform data
    sc = StandardScaler()

    X_full = adm_full[feature_cols]
    X_final = adm_full[final_features]
    y = adm[train_ycol]

    normed_data_full = pd.DataFrame(sc.fit_transform(X_full), columns = X_full.columns)
    normed_data_final = normed_data_full[final_features]
    
    clf_full=RandomForestClassifier(oob_score=True)
    clf_full.fit(normed_data_full, y)

    clf_final=RandomForestClassifier(oob_score=True)
    clf_final.fit(normed_data_final, y)



    test_features_full = feature_cols + [test_ycol]
    test_features_final = final_features + [test_ycol]
#     mci_datasets = [{'key' : 'ALL', 'data' : ADM_MCI[test_features].dropna()}, 
#                     {'key' : 'AB+', 'data' : ADM_MCI[ADM_MCI["ABP"] == 1][test_features].dropna()},
#                     {'key' : 'AB-', 'data' : ADM_MCI[ADM_MCI["ABP"] == 0][test_features].dropna()},
#                    ]  
#     for mci_dataset in mci_datasets:
    mci_df = ADM_MCI[test_features_full].dropna()
    mci_test_X = mci_df[feature_cols]
    mci_test_X = encodeCatsAndNormalize(mci_test_X)
    sc_mci = StandardScaler()
    normed_data_mci_full = pd.DataFrame(sc_mci.fit_transform(mci_test_X), columns = mci_test_X.columns)
    normed_data_mci_final = normed_data_mci_full[final_features]

    MCI_abp_true = mci_df[test_ycol]

    y_abp_pred_full = clf_full.predict(normed_data_mci_full)
    y_abp_pred_final = clf_final.predict(normed_data_mci_final)
    
    
    tb = mcnemar_table(y_target=MCI_abp_true, 
                   y_model1=y_abp_pred_full, 
                   y_model2=y_abp_pred_final)

    print(tb)
    chi2, p = mcnemar(ary=tb, corrected=True)
    if(tb[0, 1] + tb[1, 0] >= 25) :    
        chi2, p = mcnemar(ary=tb, exact=True)
        
    return chi2, p 
